In [0]:
import pandas as pd
import requests
import json

In [6]:
DATA_URL = 'https://s3.amazonaws.com/vargo.aprd6342/data/msas.csv'
df = pd.read_csv(DATA_URL)
df.head()

,CBSA,NAME,POPESTIMATE2017
0,10180,"Abilene, TX",170219
1,10420,"Akron, OH",703505
2,10500,"Albany, GA",151434
3,10540,"Albany, OR",125047
4,10580,"Albany-Schenectady-Troy, NY",886188


Knowing that we're going after Organic consumers, what average HH income should we choose? This will be your first consideration, parameter #1.

Second,  what age range should we target? This is parameter #2.

Should we look for areas with large Hispanic populations? (Parameter #3)

Now, make a python list of all the table ids you want to download. Something like this:

In [0]:
tableids = ['B01001', 'B13016', 'B13017']
tableidsstring = ",".join(tableids)

In [0]:
geoid='31000US' + df['CBSA'].astype(str)
df['geoid'] = geoid

Now, we could iterate over each of your metro areas and get the right data for each metro. To do that, we would generate a URL for each metro id and give the api the list of tables we want. The final URL would look something like this:

But there is a better way to do this. Census reporter is organized into hierarchical summary regions, with 010 representing the entire US. You can get all the data for the whole US in a single shot with a URL like this:

Note that you'll have to join your list together in a way that the api accepts, which is a list of ids separated by only commas. This handy .join() function will do just that:

In [11]:
geoidstring_part1 = ",".join(geoid[:int(len(geoid)/2)])
requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidsstring, geoidstring_part1)
loadedjson1 = requests.get(requesturl1)
parsedjson1 = json.loads(loadedjson1.text)

NameError: ignored

In [12]:
geoidstring_part2 = ",".join(geoid[int(len(geoid)/2):])
requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidsstring, geoidstring_part2)
loadedjson2 = requests.get(requesturl2)
parsedjson2 = json.loads(loadedjson2.text)

NameError: ignored

In [13]:
alldata = dict(parsedjson1['data'], **parsedjson2['data'])
alldata['31000US10180']['B03002']['estimate']

NameError: ignored

In [14]:
HH_Income = []
Pop_by_age = []
Pop_by_Hispanic = []

for i, row in df.iterrows():
    Income = alldata[df['geoid'][i]]['B19001']['estimate']['B19001011']
    + alldata[df['geoid'][i]]['B19001']['estimate']['B19001012']
    HH_Income.append(Income)
    Pop = alldata[df['geoid'][i]]['B01001']['estimate']['B01001007']
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001008']
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001009']
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001010']
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001011']
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001012']
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001031']
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001032']
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001033']
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001034']
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001035']
    + alldata[df['geoid'][i]]['B01001']['estimate']['B01001036']
    Pop_by_age.append(Pop)
    Hispanic = alldata[df['geoid'][i]]['B03002']['estimate']['B03002012']
    Pop_by_Hispanic.append(Hispanic)

df['Pop_by_Income'] = HH_Income
df['Pop_by_Age'] = Pop_by_age
df['Pop_by_Hispanic'] = Pop_by_Hispanic

NameError: ignored